In [1]:
import folium

In [6]:
seoul_map = folium.Map(location = [37.5614378, 126.9751701], zoom_start=14)
#seoul_map

In [8]:
folium.Marker([37.5614378, 126.9751701], popup='서울시청').add_to(seoul_map)

In [10]:
#seoul_map

In [11]:
import pandas as pd

In [12]:
school_path = r'공시대상학교정보_2019.xlsx'
school_raw = pd.read_excel(school_path)
school_raw.head()

,시도교육청,지역교육청,지역,정보공시 \n 학교코드,학교명,학교급코드,설립구분,학교특성,분교여부,설립유형,...,우편번호,학교도로명 \n 우편번호,학교도로명 \n 주소,학교도로명 \n 상세주소,위도,경도,전화번호,팩스번호,홈페이지 \n 주소,남녀공학 \n 구분
0,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 서초구,S000003511,서울교육대학교부설초등학교,2,국립,NaN,N,부설,...,137742,06639,서울특별시 서초구 서초중앙로 96,"(서초동,서울교대초등학교)",37.490739,127.015424,02-6009-6000,02-6009-6090,http://ps.snue.ac.kr/,남녀공학
1,서울특별시교육청,서울특별시중부교육지원청,서울특별시 종로구,S000003563,서울대학교사범대학부설초등학교,2,국립,NaN,N,부설,...,110810,03087,서울특별시 종로구 대학로 64,"(동숭동,서울사대부설초등학교)",37.577017,127.003091,02-768-1500,02-766-4085,http://www.snu.es.kr,남녀공학
2,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000738,서울개일초등학교,2,공립,NaN,N,단설,...,135804,06323,서울특별시 강남구 개포로 401,", 개일초등학교 (개포동,개일초등학교)",37.486214,127.057742,02-571-8320,02-571-8325,http://gaeil.sen.es.kr,남녀공학
3,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000741,서울구룡초등학교,2,공립,NaN,N,단설,...,135244,06305,서울특별시 강남구 개포로 263,"(개포동,구룡초등학교)",37.481193,127.051755,02-573-5017,02-575-4237,http://kuryong.sen.es.kr,남녀공학
4,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000742,서울논현초등학교,2,공립,NaN,N,단설,...,135010,06119,서울특별시 강남구 강남대로120길 33,"(논현동,논현초등학교)",37.500000,127.000000,02-514-6636,02-514-7072,http://nonhyun.sen.es.kr,남녀공학


In [13]:
school_raw.columns

Index(['시도교육청', '지역교육청', '지역', '정보공시 \n 학교코드', '학교명', '학교급코드', '설립구분', '학교특성',
       '분교여부', '설립유형', '주야구분', '개교기념일', '설립일', '법정동코드', '주소내역', '상세 \n 주소내역',
       '우편번호', '학교도로명 \n 우편번호', '학교도로명 \n 주소', '학교도로명 \n 상세주소', '위도', '경도',
       '전화번호', '팩스번호', '홈페이지 \n 주소', '남녀공학 \n 구분'],
      dtype='object')

In [14]:
school_data = school_raw[['정보공시 \n 학교코드', '학교명', '위도', '경도']]
school_data.columns = ['학교코드' ,'학교명', '위도', '경도']
# 중복 데이터 제거
school_df = school_data.drop_duplicates()

In [15]:
school_df.head()

,학교코드,학교명,위도,경도
0,S000003511,서울교육대학교부설초등학교,37.490739,127.015424
1,S000003563,서울대학교사범대학부설초등학교,37.577017,127.003091
2,S010000738,서울개일초등학교,37.486214,127.057742
3,S010000741,서울구룡초등학교,37.481193,127.051755
4,S010000742,서울논현초등학교,37.500000,127.000000


In [16]:
def graduate_preprocessing(path):
    raw_graduate = pd.read_excel(path, sheet_name = '2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '정보공시 \n 학교코드','졸업자.2', 
                           '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역''학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_date = select_col.drop(0)
    graduate_date['과고'] = pd.to_numeric(graduate_date['과고'])
    graduate_date['외고'] = pd.to_numeric(graduate_date['외고'])
    graduate_date['졸업자'] = pd.to_numeric(graduate_date['졸업자'])
    graduate_date['총합'] = graduate_date['과고'] + graduate_date['외고']
    
    def get_sido(x):
        tmp = x.split(' ')[0]
        if len(tmp) != 4:
            return tmp[:2]
        else:
            return tmp[0]+tmp[2]
    
    graduate_date['시도'] = graduate_date['지역학교명'].dropna().apply(get_sido)
    graduate_date['구군'] = graduate_date['지역학교명'].dropna().apply(lambda x:x.split(' ')[1])
    
    graduate_date[588, '시도'] = '부산'
    graduate_date[588, '구군'] = '기장군'
    graduate_date[3011, '시도'] = '경북'
    graduate_date[3011, '구군'] = '예천군'
    
    return graduate_date

In [17]:
grad_df = graduate_preprocessing(r'2018년 2차_졸업생의 진로 현황(전체).xlsx')
grad_df.head()

,지역학교명,학교코드,졸업자,과고,외고,총합,시도,구군,"(588, 시도)","(588, 구군)","(3011, 시도)","(3011, 구군)"
1,서울특별시 성북구,S000003514,234.0,4.0,5.0,9.0,서울,성북구,부산,기장군,경북,예천군
2,서울특별시 종로구,S000003562,172.0,0.0,8.0,8.0,서울,종로구,부산,기장군,경북,예천군
3,서울특별시 강남구,S010000698,214.0,2.0,9.0,11.0,서울,강남구,부산,기장군,경북,예천군
4,서울특별시 서초구,S010000700,310.0,5.0,4.0,9.0,서울,서초구,부산,기장군,경북,예천군
5,서울특별시 강남구,S010000701,256.0,3.0,4.0,7.0,서울,강남구,부산,기장군,경북,예천군


In [18]:
total_school_df = pd.merge(grad_df, school_df, how='inner', right_on ='학교코드', left_on = '학교코드')
total_school_df.head()

,지역학교명,학교코드,졸업자,과고,외고,총합,시도,구군,"(588, 시도)","(588, 구군)","(3011, 시도)","(3011, 구군)",학교명,위도,경도
0,서울특별시 성북구,S000003514,234.0,4.0,5.0,9.0,서울,성북구,부산,기장군,경북,예천군,서울대학교사범대학부설중학교,37.596201,127.039164
1,서울특별시 종로구,S000003562,172.0,0.0,8.0,8.0,서울,종로구,부산,기장군,경북,예천군,서울대학교사범대학부설여자중학교,37.577928,127.003899
2,서울특별시 강남구,S010000698,214.0,2.0,9.0,11.0,서울,강남구,부산,기장군,경북,예천군,개원중학교,37.491641,127.071415
3,서울특별시 서초구,S010000700,310.0,5.0,4.0,9.0,서울,서초구,부산,기장군,경북,예천군,경원중학교,37.510997,127.008879
4,서울특별시 강남구,S010000701,256.0,3.0,4.0,7.0,서울,강남구,부산,기장군,경북,예천군,구룡중학교,37.486153,127.056123


In [19]:
seoul_school = total_school_df[total_school_df['시도'] == '서울']
good_school = seoul_school[seoul_school['총합'] > 0]
seoul_map = folium.Map(location = [37.5614378, 126.9751701], zoom_start=11)

for n in good_school.index:
    folium.Marker([good_school['위도'][n], good_school['경도'][n]], popup = good_school['학교명'][n]).add_to(seoul_map)

In [20]:
seoul_map

In [25]:
seoul_school['비율'] = seoul_school['총합'] / seoul_school['졸업자'] *100
good_school = seoul_school[seoul_school['비율'] >= 3]
bad_school = seoul_school[seoul_school['비율'] < 3]
seoul_map = folium.Map(location = [37.5614378, 126.9751701], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([good_school['위도'][n], good_school['경도'][n]], 
                        color = 'crimson', fill_color='crimson', radius = 7).add_to(seoul_map)

for n in bad_school.index:
    folium.CircleMarker([bad_school['위도'][n], bad_school['경도'][n]], 
                        color = '#3186cc', fill_color='#3186cc', radius = 7).add_to(seoul_map)
seoul_map

/Users/soohyunson/anaconda3/envs/py-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
sido = '대전'
sido_school = total_school_df[total_school_df['시도'] == sido]
sido_school['비율'] = sido_school['총합'] / sido_school['졸업자']*100
good_school = sido_school[sido_school['비율'] >= 3]
bad_school = sido_school[sido_school['비율']<3]
sido_map = folium.Map(location = [36.350461, 127.38263], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([good_school['위도'][n], good_school['경도'][n]], 
                        color = 'crimson', fill_color='crimson', radius = 7).add_to(sido_map)

for n in bad_school.index:
    folium.CircleMarker([bad_school['위도'][n], bad_school['경도'][n]], 
                        color = '#3186cc', fill_color='#3186cc', radius = 7).add_to(sido_map)
sido_map

/Users/soohyunson/anaconda3/envs/py-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


- 등치지역도(Choropleth) : 시군구 경계 표시, 부동산 데이터 특성에 따라 지도 위에서 다르게 나타내기

- 좌표표현방식 
    - UTM-K(GR280) 좌표계 -> WGs84 타원체 경위도 좌표계 -> json file 생성


- 법정동 코드
    - 10자리 ex) 1111010100(서울특별시 종로구 청운동) -> 11110(서울특별시 종로구)

In [28]:
local_code = pd.read_excel('KIKcd_B.20190101.xlsx')
local_code['법정동코드'] = local_code['법정동코드'].apply(lambda x:str(x)[:5])
filtered_code = local_code[['법정동코드', '시도명', '시군구명']].drop_duplicates()
filtered_code.dropna(inplace=True)

In [40]:
filtered_code.loc[20477] = ('36110', '세종', '세종')

In [41]:
def get_sido(x):
    tmp = x.split(' ')[0]
    if len(tmp) != 4:
        return tmp[:2]
    else:
        return tmp[0] + tmp[2]
filtered_code['시도명'] = filtered_code['시도명'].dropna().apply(get_sido)
filtered_code['시군구명']=filtered_code['시군구명'].dropna().apply(lambda x: x.split(' ')[-1])

In [42]:
from utils import *

In [43]:
price_index = KBpriceindex_preprocessing('KB_ts.xls', '매매apt')

In [44]:
diff_pct = ((price_index.loc['2018-6-1']/price_index.loc['2017-6-1']) -1) * 100
diff_df = pd.DataFrame({'증감률' : diff_pct})

In [46]:
filtered_code.index = [filtered_code['시도명'], filtered_code['시군구명']]

code = []
for local in diff_df.index:
    if local[0] in filtered_code.index:
        tmp_df = filtered_code.loc[local[0]]
        if local[1] in tmp_df.index:
            code.append(tmp_df.loc[local[1]]['법정동코드'])
        elif local[1] + '시' in tmp_df.index:
            code.append(tmp_df.loc[local[1] + '시']['법정동코드'])
        elif local[1] == '세종':
            code.append('36110')
        else:
            code.append('')
    else:
        code.append('')

diff_df['법정동코드'] = code

In [48]:
import json

file = open('TL_SCCO_SIG_WGS84.json', 'r', encoding='utf-8').read()
jsonData = json.loads(file)

In [53]:
korea_map = folium.Map(location=[36, 127], zoom_start=7)

korea_map.choropleth(
    # geo_data ; 지도 위의 경계선, 도형에 관한 데이터를 담은 변수를 연결
    geo_data = jsonData,
    # data ; 경계선, 도형과 연관된 데이터를 연결
    data=diff_df,
    # data에서 geo_data와 연결되는 column 이름 (geo_data와 연결되는 값을 가진 column을 먼저 명시해야 함.)
    columns=['법정동코드', '증감률'],
    # geo_data에서 data와 연결해줄 property를 입력. 
    key_on = 'feature.properties.SIG_CD',
    fill_color = 'RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='증감률(%)'
)

In [56]:
#korea_map

In [55]:
korea_map.save('부동산 증감 지도.html')